# Explore the statistics across all scenarios

This notebook aims at providing basic tools for exploring the results of the rule `run_all_scenarios`.
After loading all files `results/*/stats.csv`, the notebooks explore the content

### Load imports

In [ ]:
import xarray as xr
import geopandas as gpd
import pandas as pd
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import requests
import pypsa
import shutil
from rasterio.plot import show
from shapely.wkt import loads
from shapely.geometry import Point
from shapely.geometry import LineString
from pathlib import Path
import country_converter as coco


# change current directory to parent folder
import os
import sys

if not os.path.isdir("pypsa-earth"):
    os.chdir("../..")
sys.path.append(os.getcwd()+"/pypsa-earth/scripts")

from _helpers import read_csv_nafix
from _helpers import two_digits_2_name_country, country_name_2_two_digits

cc = coco.CountryConverter()

## Load all stats.csv files

Loads all files in the form of `results/{scenario_name}/stats.csv` and creates a dataframe indexed by `scenario_name`.
The merged dataframe is then saved locally.

In [ ]:
import pandas as pd

df_list = []

for scenario in Path("pypsa-earth/results").glob("*"):
    if scenario.is_dir():
        stats_path = Path.joinpath(scenario, "stats.csv")
        if stats_path.is_file():
            df_temp = read_csv_nafix(str(stats_path), header=[0, 1], index_col=0)
            df_temp.index = [str(scenario.name)]
            df_list.append(df_temp)
        else:
            print("Missing entry for " + str(stats_path))

stats = pd.concat(df_list)
stats.to_csv("documentation/stats_merged.csv")

## Country-wise analysis

The following section focuses on analyzing the results of a country-wise analysis.

### Filter counry-specific scenarios

In [ ]:
# filter country-specific scenarios
stats_country = stats[stats.index.str.len() <= 2].copy()

In [ ]:
# add ISO3 country code for later analyses
stats_country["iso_a3"] = cc.convert(names=stats_country.index, to='ISO3')

### Filter the UN countries

In [ ]:
# Check what 2-letter country codes are missing from the UN list
# the list is then stored in missing_codes
UN_2_code = coco.convert(cc.UN.name_short, to="ISO2")
missing_codes = set(UN_2_code) - set(stats_country.index)
missing_codes_name_short = coco.convert(list(missing_codes), to="name_short")
print("The missing UN countries are as follows:")
dict(zip(missing_codes, missing_codes_name_short))

In [ ]:
UN_stats = stats_country.loc[stats_country.index.intersection(UN_2_code)]

total_pop = UN_stats[('build_shapes', 'pop')].sum()

print(("Total population: {:.2f}".format(total_pop/1e9)))

total_pop_working = UN_stats.loc[UN_stats[('snakemake_status', 'solve_network')], ('build_shapes', 'pop')].sum()

print(("Total population working: {:.2f}".format(total_pop_working/1e9)))
print("Working represented country [%]: {:.2f}%%".format(total_pop_working/total_pop*100))

## Plot the status of the workflow by country

Collect in the column status_level, the number of rules that have succeded

In [ ]:
stats["status_level"] = stats[("snakemake_status", "solve_network")]
stats["status_level"].sum()/stats.shape[0]

#### Missing countries

In [ ]:
stats[~stats["status_level"]].index

In [ ]:
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.cm as cm

def custom_cmap(x):

    if x[("snakemake_status", "solve_network")] == True:  # till solved network
        return (0, 1, 0)  # green
    if x[("snakemake_status", "simplify_network")] == True:  #
        return (0, .5, 0)  # dark green
    
    return (1.0, 0.8, 0)  # orange
    # if x[("snakemake_status", "base_network")] == True:
    #     return (1, 1, 0)  # yellow
    # if x[("snakemake_status", "build_shapes")] == True:
    #     return (1.0, 0.6, 0)  # orange
    # return (1, 0, 0)  # red

plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

countries = gpd.read_file(
               gpd.datasets.get_path("naturalearth_lowres")).set_index("iso_a3")

stats_plot = stats_country.set_index("iso_a3")

exclude_countries = [] #["USA", "CAN", "MEX", "RUS", "GRL",  "TLS"] #"IDN",
exclude_continents = [] #["Europe", "Oceania"]

count_countries = 0

for c_iso_a3, row in countries.iterrows():
    # if c_iso_a3 in exclude_countries or coco.convert(c_iso_a3, to="Continent") in exclude_continents:
    #     continue
    if c_iso_a3 in stats_plot.index:
        # if not stats_plot.loc[c_iso_a3,("snakemake_status", "solve_network")]:
        #     continue
        ax.add_geometries(row["geometry"], ccrs.PlateCarree(),
                          facecolor=custom_cmap(stats_plot.loc[c_iso_a3]),
                          label=c_iso_a3)
        count_countries += stats_plot.loc[c_iso_a3, ("snakemake_status", "solve_network")]
        # if stats.loc[country.attributes['ISO_A2'], ("snakemake_status", "solve_network")]:
        #     ax.add_geometries(country.geometry, ccrs.PlateCarree(),
        #                     facecolor=(0, 1, 0),
        #                     label=country.attributes['ISO_A2'])


plt.savefig("documentation/img.png")
plt.show()

In [ ]:
import pandas as pd
import matplotlib


df_marginal_price = pd.DataFrame(columns=["avg_marginal", "total_demand"])

for scenario in Path("pypsa-earth/results").glob("*"):
    if scenario.is_dir():
        n_path = Path.joinpath(scenario, "networks/elec_s_20flex_ec_lcopt_1H.nc")
        if n_path.is_file():
            n = pypsa.Network(n_path)
            print(n_path)
            common_cols = n.buses_t.marginal_price.columns.intersection(n.loads_t.p_set.columns)
            total_demand = n.loads_t.p_set[common_cols].sum().sum()+0.0000001
            avg_marginal = (n.loads_t.p_set[common_cols] * n.buses_t.marginal_price[common_cols]).sum().sum()/total_demand
            df_marginal_price.loc[scenario.name] = [avg_marginal, total_demand]
        else:
            print("Missing entry for " + str(n_path))

df_marginal_price.to_csv("documentation/marginal_price.csv")

In [ ]:

import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.cm as cm

import country_converter as coco

df_marginal_price["iso_a3"] = coco.convert(names=df_marginal_price.index, to='ISO3')
df_marginal_price = df_marginal_price.set_index("iso_a3")

def custom_cmap(x):

    if x[("snakemake_status", "solve_network")] == True:  # till solved network
        return (0, 1, 0)  # green
    if x[("snakemake_status", "simplify_network")] == True:  #
        return (0, .5, 0)  # dark green
    
    return (1.0, 0.8, 0)  # orange
    # if x[("snakemake_status", "base_network")] == True:
    #     return (1, 1, 0)  # yellow
    # if x[("snakemake_status", "build_shapes")] == True:
    #     return (1.0, 0.6, 0)  # orange
    # return (1, 0, 0)  # red

fig = plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

countries = gpd.read_file(
               gpd.datasets.get_path("naturalearth_lowres")).set_index("iso_a3")

exclude_countries = [] #["USA", "CAN", "MEX", "RUS", "GRL",  "TLS"] #"IDN",
exclude_continents = [] #["Europe", "Oceania"]

cmap = plt.get_cmap('viridis')

norm = matplotlib.colors.Normalize(vmin=df_marginal_price.avg_marginal.min(), vmax= 200) #df_marginal_price.avg_marginal.max())

df_marginal_price["color"] = df_marginal_price.avg_marginal.map(norm)

for c_iso_a3, row in countries.iterrows():
    # if c_iso_a3 in exclude_countries or coco.convert(c_iso_a3, to="Continent") in exclude_continents:
    #     continue
    if c_iso_a3 in df_marginal_price.index:
        # if not stats_plot.loc[c_iso_a3,("snakemake_status", "solve_network")]:
        #     continue
        ax.add_geometries(row["geometry"], ccrs.PlateCarree(),
                        facecolor=cmap(df_marginal_price.loc[c_iso_a3, "color"]),
                          label=c_iso_a3)
        # if stats.loc[country.attributes['ISO_A2'], ("snakemake_status", "solve_network")]:
        #     ax.add_geometries(country.geometry, ccrs.PlateCarree(),
        #                     facecolor=(0, 1, 0),
        #                     label=country.attributes['ISO_A2'])

sm = plt.cm.ScalarMappable(cmap=cmap, norm = norm)
sm._A = []
cb = plt.colorbar(sm, ax = ax, label="[€/MWh]")
plt.title("Average marginal price")

# plt.savefig("documentation/img.png")
# plt.show()

In [ ]:
df_marginal_price.sort_values("avg_marginal").to_csv("documentation/sorted.csv")

In [ ]:
df_marginal_price.sort_values("avg_marginal")["avg_marginal"].plot()

In [ ]:
df_marginal_price.sort_values("avg_marginal", ascending=False)

In [ ]:
stats_plot.columns.get_level_values(0).unique()

In [ ]:
stats_plot["snakemake_status"]["solve_network"].sum()

In [ ]:
stats_plot["total_comp_stats"]["total_time"].sum()/stats_plot["snakemake_status"]["solve_network"].sum()

In [ ]:

# import cartopy
# import cartopy.io.shapereader as shpreader
# import cartopy.crs as ccrs
# import cartopy.feature as cf
# import matplotlib.cm as cm

# def custom_cmap(x):
#     if x[("snakemake_status", "solve_network")] == True:  # till solved network
#         return (0, 1, 0)  # green
    
#     print("Non-green element: " + str(x.name))

#     if x[("snakemake_status", "simplify_network")] == True:  #
#         return (0, .5, 0)  # dark green
#     if x[("snakemake_status", "base_network")] == True:
#         return (1, 1, 0)  # yellow
#     if x[("snakemake_status", "build_shapes")] == True:
#         return (1.0, 0.6, 0)  # orange
#     return (1, 0, 0)  # red

# plt.figure(figsize=(5, 3), dpi=300)
# ax = plt.axes(projection=ccrs.PlateCarree())
# #ax.add_feature(cf.LAND)
# #ax.add_feature(cf.OCEAN)
# ax.add_feature(cf.COASTLINE)
# # ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
# #ax.add_feature(cf.LAKES, alpha=0.95)
# #ax.add_feature(cf.RIVERS)
# ax.add_feature(cf.BORDERS)
# # ax.set_extent([-150, 60, -25, 60])

# shpfilename = shpreader.natural_earth(resolution='110m',
#                                       category='cultural',
#                                       name='admin_0_countries')
# reader = shpreader.Reader(shpfilename)
# countries = reader.records()

# for country in countries:
#     if country.attributes['ISO_A2'] in stats.index :
#         ax.add_geometries(country.geometry, ccrs.PlateCarree(),
#                           facecolor=custom_cmap(stats.loc[country.attributes['ISO_A2']]),
#                           label=country.attributes['ISO_A2'])
#         # if stats.loc[country.attributes['ISO_A2'], ("snakemake_status", "solve_network")]:
#         #     ax.add_geometries(country.geometry, ccrs.PlateCarree(),
#         #                     facecolor=(0, 1, 0),
#         #                     label=country.attributes['ISO_A2'])


# plt.savefig("documentation/img.png")
# plt.show()

## Plot the computational requirements 

In the following, the computational statistics by scenario are plotted and analyzed

In [ ]:
sel_cols = ["total_time", "max_memory"]

# stats_comp = stats.loc[:, stats.columns.get_level_values(1).isin(sel_cols)].transpose()
# stats_comp.index = stats_comp.index.reorder_levels([1,0])
# stats_comp.columns = ["USa", "USb", "Europe", "SouthAmerica", "Africa"]

stats_comp = stats.loc[:, stats.columns.get_level_values(1).isin(sel_cols)].iloc[2:].transpose()
stats_comp.index = stats_comp.index.reorder_levels([1,0])
stats_comp.columns = ["Europe", "SouthAmerica", "Africa"]

for kn, gv in stats_comp.groupby(stats_comp.index.get_level_values(0)):
    gv_plot = gv.copy()
    gv_plot.index = gv_plot.index.droplevel(0)


    title_val = kn
    if kn == "max_memory":
        title_val = "max_memory (Gb)"
        gv_plot = gv_plot/1000
        # gv_plot = gv_plot.groupby(gv_plot.index).max()
    elif kn == "total_time":
        title_val = "total_time (h)"
        gv_plot = gv_plot/3600
        # gv_plot = gv_plot.groupby(gv_plot.index).sum()

    gv_plot.plot(kind="bar", title=title_val,logy=True)

### Other plotting and analyses

In [ ]:
import cartopy
import cartopy.io.shapereader as shpreader
import cartopy.crs as ccrs
import cartopy.feature as cf
import matplotlib.cm as cm

def custom_cmap_original(x):
    if x == "NG":  # till solved network
        return (0, 1, 0)  # green
    return (0.2, 0.2, 1)  # blue

plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

for country in countries:
    if country.attributes['CONTINENT'] == "Africa":
        ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                          facecolor=custom_cmap_original(country.attributes['ISO_A2']),
                          label=country.attributes['ISO_A2'])


plt.savefig("documentation/img_original.png")
plt.show()

In [ ]:
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))

In [ ]:
world.keys()

In [ ]:
plt.figure(figsize=(5, 3), dpi=300)
ax = plt.axes(projection=ccrs.PlateCarree())
#ax.add_feature(cf.LAND)
#ax.add_feature(cf.OCEAN)
ax.add_feature(cf.COASTLINE)
# ax.add_feature(cf.BORDERS, linestyle='-', alpha=.5)
#ax.add_feature(cf.LAKES, alpha=0.95)
#ax.add_feature(cf.RIVERS)
ax.add_feature(cf.BORDERS)
# ax.set_extent([-150, 60, -25, 60])

shpfilename = shpreader.natural_earth(resolution='110m',
                                      category='cultural',
                                      name='admin_0_countries')
reader = shpreader.Reader(shpfilename)
countries = reader.records()

for country in countries:
    if country.attributes['ISO_A2'] in stats[stats[("snakemake_status", "add_electricity")] == False].index:
        print(country.attributes['ISO_A2'])
        ax.add_geometries(country.geometry, ccrs.PlateCarree(),
                          facecolor=(1, 0, 0),
                          label=country.attributes['ISO_A2'])


plt.savefig("documentation/img_not_working.png")
plt.show()

In [ ]:
# # This code is useful to remove the output files of non-working scenarios to repeat them
# # commented out by default
 
# import pandas as pd

# df_list = []

# for c in stats.index[stats.loc[:, ("snakemake_status", "solve_network")]==False]:

#     if c not in UN_2_code:
#         continue
#     stat_path = os.getcwd() + f"/pypsa-earth/results/{c}/stats.csv"
#     # # print(stat_path)
#     # if not os.path.isfile(stat_path):
#     #     print(f"SKIP {c}")
#     #     # continue
    
#     # os.remove(os.getcwd() + f"/pypsa-earth/results/{c}/scenario.done")
#     # os.remove(os.getcwd() + f"/pypsa-earth/results/{c}/stats.csv")

#     for d in ["results", "resources", "networks"]:
#         dirpath = os.getcwd() + f"/pypsa-earth/{d}/{c}/"
#         if os.path.isdir(dirpath):
#             # print(dirpath)
#             shutil.rmtree(dirpath)

In [ ]:
import geopandas as gpd

In [ ]:
g = gpd.read_file("/data/davidef/gitsegan/pypsa-earth/resources/IN/osm/raw/all_raw_lines.geojson")
g.geom_type.unique()

In [ ]:
g.geometry.map(lambda g: len(g.boundary.geoms) >= 2)

In [ ]:
def true_on_error(x):
    try:
        len(x.boundary.geoms)
        return False
    except:
        return True

In [ ]:
e = g.geometry.map(lambda x: true_on_error(x))

In [ ]:
g.geom_type.unique()

In [ ]:
print(g.iloc[0].geometry)

In [ ]:
print(g[e].iloc[0].geometry)

In [ ]:
g[e].to_crs("EPSG:3857").area